In [1]:
import math

import sys
sys.path.insert(0, '..')
#import d2l
import torch
import torch.nn as nn
import torch.optim as optim
import time
import numpy
import yacht_main as yacht

In [2]:
INPUT_SIZE = 6
OUTPUT_SIZE = 12

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(INPUT_SIZE)
    
net = torch.nn.Sequential(
    Reshape(),
    
    nn.Linear(in_features=INPUT_SIZE, out_features=50),
    nn.LeakyReLU(),
    nn.Linear(50,30),
    nn.LeakyReLU(),
    nn.Linear(30,OUTPUT_SIZE)
    
    #nn.Linear(in_features=INPUT_SIZE, out_features = OUTPUT_SIZE)
)

def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.uniform_(m.weight,-0.5,0.5)

net = net.float()
net.apply(init_weights)

Sequential(
  (0): Reshape()
  (1): Linear(in_features=6, out_features=50, bias=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): Linear(in_features=50, out_features=30, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Linear(in_features=30, out_features=12, bias=True)
)

In [3]:
def try_gpu():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

device = try_gpu()
print(torch.cuda.is_available())
net = net.to(device)

True


In [4]:
train_data = []
SCALE = 30.0


def pack(org):
    return (org/SCALE)

def unpack(norm):
    if math.isnan(norm):
        return float(100)
    return round(SCALE*float(norm))
"""
for d1 in range(6):
    for d2 in range(6):
        for d3 in range(6):
            for d4 in range(6):
                for d5 in range(6):
                    dices = [d1+1,d2+1,d3+1,d4+1,d5+1]
                    scores = [pack(f(dices)) for f in yacht.score_func]
                    #scores = [yacht.score_func[6](dices)/SCALE]
                    
                    dt = torch.tensor([dices.count(i+1)/5.0 for i in range(6)], dtype = torch.float32, requires_grad = False)
                    st = torch.tensor(scores, dtype = torch.float32, requires_grad = False)
                    
                    train_data.append((dt,st))
"""

for c1 in range(6):
    for c2 in range(6-c1):
        for c3 in range(6-c1-c2):
            for c4 in range(6-c1-c2-c3):
                for c5 in range(6-c1-c2-c3-c4):
                    c6 = 5-c1-c2-c3-c4-c5
                    dices = [1]*c1 + [2]*c2 + [3]*c3 + [4]*c4 + [5]*c5 + [6]*c6

                    scores = [pack(f(dices)) for f in yacht.score_func]
                    #scores = [yacht.score_func[6](dices)/SCALE]

                    dt = torch.tensor([c1,c2,c3,c4,c5,c6], dtype = torch.float32, requires_grad = False)
                    st = torch.tensor(scores, dtype = torch.float32, requires_grad = False)

                    train_data.append((dt/5.0,st))

print(str(len(train_data)))

for x,y in train_data:
    cont = ""
    for a in x:
        cont += str(int(a*5))+" "
    cont += " | "
    for b in y:
        cont += str(unpack(b))+" "
    print(cont)


252
0 0 0 0 0 5  | 0 0 0 0 0 30 30 30 30 0 0 50 
0 0 0 0 1 4  | 0 0 0 0 5 24 29 29 0 0 0 0 
0 0 0 0 2 3  | 0 0 0 0 10 18 28 0 28 0 0 0 
0 0 0 0 3 2  | 0 0 0 0 15 12 27 0 27 0 0 0 
0 0 0 0 4 1  | 0 0 0 0 20 6 26 26 0 0 0 0 
0 0 0 0 5 0  | 0 0 0 0 25 0 25 25 25 0 0 50 
0 0 0 1 0 4  | 0 0 0 4 0 24 28 28 0 0 0 0 
0 0 0 1 1 3  | 0 0 0 4 5 18 27 0 0 0 0 0 
0 0 0 1 2 2  | 0 0 0 4 10 12 26 0 0 0 0 0 
0 0 0 1 3 1  | 0 0 0 4 15 6 25 0 0 0 0 0 
0 0 0 1 4 0  | 0 0 0 4 20 0 24 24 0 0 0 0 
0 0 0 2 0 3  | 0 0 0 8 0 18 26 0 26 0 0 0 
0 0 0 2 1 2  | 0 0 0 8 5 12 25 0 0 0 0 0 
0 0 0 2 2 1  | 0 0 0 8 10 6 24 0 0 0 0 0 
0 0 0 2 3 0  | 0 0 0 8 15 0 23 0 23 0 0 0 
0 0 0 3 0 2  | 0 0 0 12 0 12 24 0 24 0 0 0 
0 0 0 3 1 1  | 0 0 0 12 5 6 23 0 0 0 0 0 
0 0 0 3 2 0  | 0 0 0 12 10 0 22 0 22 0 0 0 
0 0 0 4 0 1  | 0 0 0 16 0 6 22 22 0 0 0 0 
0 0 0 4 1 0  | 0 0 0 16 5 0 21 21 0 0 0 0 
0 0 0 5 0 0  | 0 0 0 20 0 0 20 20 20 0 0 50 
0 0 1 0 0 4  | 0 0 3 0 0 24 27 27 0 0 0 0 
0 0 1 0 1 3  | 0 0 3 0 5 18 26 0 0 0 0 0 
0 0

In [18]:
PRIORITY = 3
saveidx = 0
def train(net, train_iter,criterion, num_epochs, device,lr=None, weight_decay = 0):

    print('training on', device)
    net.to(device)
    #optimizer = optim.Adam(net.parameters(), lr=lr, weight_decay = weight_decay)
    optimizer = optim.SGD(net.parameters(), lr=lr, weight_decay = weight_decay)
    
    for epoch in range(num_epochs):
        #train_l_sum = torch.tensor([0.0], dtype=torch.float32, device=device)
        #train_loss_sum = torch.tensor([0.0], dtype=torch.float32, device=device)
        train_l_sum = 0.0
        train_loss_sum = 0.0
        
        
        n, start = 0, time.time()

        for x, y in train_iter:
            net.train()
            optimizer.zero_grad()
            x,y = x.to(device),y.to(device)
            
            y_hat = net(x)
            loss = criterion(y[:7], y_hat[:7]) + PRIORITY * criterion(y[7:],y_hat[7:])
            #loss = criterion(torch.atanh(y), torch.atanh(y_hat))
            loss.backward()
            optimizer.step()
            
            with torch.no_grad():
                train_l_sum += loss.float()
                train_loss_sum += sum([abs(unpack(d)) for d in (y-y_hat)])
                n += 1

        print('epoch %d, loss %.4f, train loss %.3f, time %.1f sec, n=%d' %
              (epoch + 1, 100.0*train_l_sum/n*SCALE, train_loss_sum/n, time.time() - start, n) )
        
    saveidx+=1
    torch.save(net.state_dict(), './data/HM_learn_scores_' + str(saveidx))
    return

In [19]:
criterion = nn.MSELoss(reduction = 'sum')
train(net, train_data, criterion,100, device, 0.001)

training on cuda:0
epoch 1, loss 3.1316, train loss 0.500, time 0.8 sec, n=252
epoch 2, loss 3.1315, train loss 0.500, time 0.7 sec, n=252
epoch 3, loss 3.1304, train loss 0.500, time 0.7 sec, n=252
epoch 4, loss 3.1283, train loss 0.500, time 0.7 sec, n=252
epoch 5, loss 3.1299, train loss 0.500, time 0.8 sec, n=252
epoch 6, loss 3.1291, train loss 0.500, time 0.9 sec, n=252
epoch 7, loss 3.1292, train loss 0.500, time 1.1 sec, n=252
epoch 8, loss 3.1281, train loss 0.500, time 0.9 sec, n=252
epoch 9, loss 3.1280, train loss 0.500, time 0.7 sec, n=252
epoch 10, loss 3.1272, train loss 0.496, time 0.7 sec, n=252
epoch 11, loss 3.1253, train loss 0.496, time 0.7 sec, n=252
epoch 12, loss 3.1269, train loss 0.496, time 0.7 sec, n=252
epoch 13, loss 3.1246, train loss 0.496, time 0.7 sec, n=252
epoch 14, loss 3.1270, train loss 0.496, time 0.7 sec, n=252
epoch 15, loss 3.1258, train loss 0.496, time 0.7 sec, n=252
epoch 16, loss 3.1234, train loss 0.496, time 0.7 sec, n=252
epoch 17, loss

UnboundLocalError: local variable 'saveidx' referenced before assignment

In [20]:
torch.save(net.state_dict(), './data/HM_learn_scores_' + str(1))

In [8]:
net.load_state_dict(torch.load('./data/HM_learn_scores_1'))

RuntimeError: [enforce fail at ..\caffe2\serialize\inline_container.cc:209] . file not found: archive/data/1738475360880

In [21]:
corr = 0

for x,y in train_data:
    net.eval()
    x,y = x.to(device),y.to(device)
    y_hat = net(x)

    hy = y.clone().detach()
    hyh = y_hat.clone().detach()
    
    dh = (hy-hyh)
    
    flag = False

    
    for i in range(OUTPUT_SIZE):
        if not unpack(dh[i]) == 0:
            flag = True
            break
    
    if not flag:
        corr += 1
        continue
    
    cont = ""
    for a in x:
        cont += str(int(a*5)) + " "
    cont += " | "
    for d in hy-hyh:
    #for d in hyh:
        cont += str(unpack(d)) + " "
    print(cont)
        
print("{0}".format(corr))


0 0 0 0 4 1  | 0 1 0 0 0 0 0 0 0 0 0 0 
0 0 0 1 0 4  | 0 0 0 0 0 1 1 0 0 0 0 0 
0 0 0 2 2 1  | 0 0 0 0 1 0 1 0 0 0 0 0 
0 0 0 2 3 0  | 0 0 0 0 1 0 0 0 0 0 0 0 
0 0 0 3 2 0  | 0 0 0 -1 2 0 1 0 0 0 0 0 
0 0 1 0 0 4  | 0 0 1 0 0 -1 -1 0 0 0 0 0 
0 0 1 2 2 0  | 0 0 0 0 2 0 1 0 0 0 0 0 
0 0 1 3 1 0  | 0 0 0 0 -1 0 0 0 0 0 0 0 
0 0 2 2 1 0  | 0 0 1 0 0 0 0 0 0 0 0 0 
0 0 2 3 0 0  | 0 0 1 0 0 0 0 0 0 0 0 0 
0 0 3 0 1 1  | 0 0 0 0 1 0 1 0 0 0 0 0 
0 0 3 0 2 0  | 0 0 0 0 -1 0 -1 0 0 0 0 0 
0 0 3 1 0 1  | 0 -1 0 0 0 0 0 0 0 0 0 0 
0 0 3 2 0 0  | 0 0 1 0 0 0 0 0 0 0 0 0 
0 0 4 0 0 1  | 0 -1 0 1 0 0 0 0 0 0 0 0 
0 0 4 0 1 0  | 0 0 -1 0 0 0 0 0 0 0 0 0 
0 0 4 1 0 0  | 0 -1 0 1 0 0 0 0 0 0 0 0 
0 0 5 0 0 0  | 0 0 -1 0 1 0 0 0 0 0 0 0 
0 1 0 1 3 0  | 0 -1 0 0 0 0 0 0 0 0 0 0 
0 1 0 2 2 0  | 0 0 0 0 1 0 0 0 0 0 0 0 
0 1 2 0 2 0  | 0 0 0 0 -1 0 0 0 0 0 0 0 
0 1 2 2 0 0  | 0 0 1 0 0 0 0 0 0 0 0 0 
0 1 3 0 1 0  | 0 0 1 -1 0 0 0 0 0 0 0 0 
0 1 3 1 0 0  | 0 0 1 0 0 0 0 0 0 0 0 0 
0 1 4 0 0 0  | 0 1 0 -2 0 

In [ ]:
net.

In [39]:
net.eval()
net(torch.tensor([0,0,0,0,0,5], dtype = torch.float32, device = device))

RuntimeError: CUDA error: an illegal memory access was encountered

In [16]:
saveidx

NameError: name 'saveidx' is not defined